# Wie Maschinen Sprache lernen: Textgeneration

Hier coden wir einen simplen Textgenerator. Ausgehend von ein paar Wörtern als Input (z.&nbsp;B. "gegen ende des") wird er neue Wörter erzeugen, die eine mögliche (bzw. wahrscheinliche) Weiterführung des Textes darstellen (hier z.&nbsp;B. "[gegen ende des] jahrhunderts könnte die erde laut aktuellem bericht gut 45 prozent" und so weiter). Solche oder ähnliche Algorithmen finden wir etwa bei Vervollständigungsvorschlägen, wenn wir auf unserem Handy tippen oder bei Google eine Suchanfrage eingeben. Der verwendete Code wurde teils von [Usman Malik](https://stackabuse.com/python-for-nlp-developing-an-automatic-text-filler-using-n-grams/) übernommen.

Wir verwenden die Programmiersprache Python für unseren Textgenerator. Sämtlicher Code ist bereits fertig geschrieben und wartet nur auf Deine Ausführung. Kommentare – alles was hinter einem `#` in einer Code-Zelle steht – erläutern Dir den Code. Wenn etwas unklar bleibt, frag den/die Lehrende(n), Deine Nachbarin oder warum nicht chatGPT?

Zunächst importieren wir ein paar Module. Führe die folgende Code-Zelle aus, indem Du in sie reinklickst und anschließend `Shift + Enter` drückst. Diesen Befehl sollst Du ab sofort bei jeder Code-Zelle ausführen.

In [ ]:
import random
import nltk
import re
from tqdm import tqdm
nltk.download('punkt')

Damit unser Algorithmus Vorhersagen treffen kann, welche Wörter wahrscheinlich auf bestimmte vorangehende Wörter folgen, müssen wir ihm ein Bild menschlicher Sprache (in unserem Fall: Deutsch) vermitteln. Das Bild sollte möglichst umfassend sein. Deshalb nutzen wir einen Datensatz von [Wortschatz Leipzig](https://wortschatz.uni-leipzig.de/de) mit 1 Million Sätzen aus deutschsprachigen Zeitungsartikeln aus dem Jahr 2022.

## 1. Daten einlesen

Beginnen wir, indem wir den Datensatz in der nächsten Code-Zelle von einem öffentlichen Cloud-Speicher herunterladen. Führe die Zelle wiederum mittels `Shift + Enter` aus.

In [ ]:
!gdown 1dct9Eo0W697cf4Tv3cj1DfAAvThwC9u1 #Download der Trainingsdaten von einer öffentlichen Google Drive

Im Dateimanager in der linken Seitenleiste sollte sich nun die Datei "data_1M.txt" befinden. Wenn Du sie per Doppelklick öffnest, siehst Du, dass es sich um eine Art Tabelle handelt, mit je einer Zahl sowie einem vollständigen Satz pro Zeile.

In der folgenden Code-Zelle lesen wir die Datei in den Arbeitsspeicher ein. Wir erstellen ganz einfach eine lange Liste mit Wörtern in der Reihenfolge, wie sie in den einzelnen Sätzen vorkommen.

In [ ]:
#Öffnen der Datei
with open("data_1M.txt") as read_file:
    
    words = [] #Erstellen einer leeren Liste, an die unten Wort für Wort angehängt wird
    
    #Iterieren über die geöffnete Datei, d. h. "Aufrufen" einer Zeile nach der anderen
    #"tdqm" sorgt für eine Fortschrittsanzeige während der Iteration
    for line in tqdm(read_file):
        
        #Trennen (Splitten) der Zeile bei jedem Tabulator (\t), der zwischen Zahl und Satz steht
        #Zugriff auf das zweite dabei entstehende Element (also den Satz) mittels [1] (1, da Python bei 0 zu zählen beginnt!)
        sentence = line.split("\t")[1]
        
        #Bereinigen des Satzes, indem überflüssige Leerzeichen entfernt (strip),
        #sämtliche Buchstaben kleingeschrieben (lower) sowie diverse Sonderzeichen entfernt werden (re.sub)
        sentence_preprocessed = sentence.strip().lower()
        sentence_preprocessed = re.sub(r"[^A-Za-z0-9ÄäÖöÜüß.,?!\s\-]", "", sentence_preprocessed)
        
        words_per_sentence = nltk.word_tokenize(sentence_preprocessed) #Trennen (Splitten) des Satzes bei jedem Leerzeichen, d.h. Unterteilen nach Wörtern
        
        words.extend(words_per_sentence) #Anfügen von "words_per_sentence" an "words"

Inspizieren wir mal die eingelesenen Daten und lassen uns die ersten 100 Wörter ausgeben:

In [ ]:
print(words[0:100]) #Über die eckigen greifen wir auf die ersten 100 Wörter zu

Lass Dir gerne einen anderen Ausschnitt der Wortliste ausgeben, indem Du die "Start-" und "Endzahl" des Ausschnittes innerhalb der eckigen Klammern anpasst. Würdest Du die eckigen Klammern entfernen, so gäbe Python die gesamte Wortliste aus. Das könnte Deinen Rechner aber überfordern, da die Liste aus über 15 Millionen Wörtern besteht, wie die nächste Code-Zelle errechnet:

In [ ]:
#Ausgabe der Länge (len) der Wortliste
len(words)

Unser Textgenerator basiert auf sog. *n-Grammen*. N-Gramme sind Sequenzen der Länge *n*. Ein Trigramm etwa besteht aus jeweils drei aufeinanderfolgenden Einheiten. In unserem Fall sind Wörter die Einheit (in anderen Zusammenhängen kann die Einheit aber auch was ganz anderes sein, z.&nbsp;B. Basen [A, C, G, T] bei der Vorhersage von DNA-Strängen). Dieser Textabschnitt etwa beginnt mit dem Wort-Trigramm "Unser Textgenerator basiert", gefolgt von "Textgenerator basiert auf", "basiert auf sog." usw. 

Wort-n-Gramme, insbesondere Wort-Trigramme, eignen sich viel besser, um Vorhersagen über folgende Wörter zu treffen, als ein einzelnes Wort, da menschliche Sprache hochgradig musterhaft ist und sich diese Muster typischerweise über mehrere Wörter hinweg manifestieren. Konkret: Auf das einzelne Wort "gegen" können sehr viele verschiedene Wörter folgen, auf "gegen ende" schon weniger und bei "gegen ende des" machen nicht mehr allzu viele Folgewörter Sinn. 

Entscheidend ist, dass unser Algorithmus wird nicht nur lernen wird, welche Wörter auf welche Wort-Trigramme in den Trainingsdaten folgen, sondern auch, wie häufig ein bestimmtes Wort auf ein gegebenes Wort-Trigramm folgt. Bei "gegen ende des" vom Anfang folgt in unseren Trainingsdaten z.&nbsp;B. in 20% der Fälle das Wort "jahres", in 11% das Wort "zweiten" etc. Unser Algorithmus wird für sämtliche Wort-Trigramme alle darauffolgenden Wörter inkl. ihrer relativen Häufigkeit lernen und dadurch in der Lage sein, neuen Text zu generieren.

## 2. Algorithmus trainieren

Der nächste Schritt ist das Herzstück des Codes, in dem wir die oben beschriebene Idee implementieren. Zunächst definieren wir ein leeres sog. *dictionary*, das man sich wie ein reales Wörterbuch vorstellen kann. Abstrakt gesehen besteht es also aus Schlüssel-Wert-Paaren. Bei einem Bedeutungswörterbuch entspräche der Schlüssel dem jeweiligen Wort und der zugehörige Wert dessen Bedeutung(en). 

In unserem Fall schaffen wir für jedes Wort-Trigram, das in den Trainingsdaten vorkommt, einen Schlüssel. Der zugehörige Wert wiederum ist eine Liste mit Wörtern, die jeweils direkt nach dem Wort-Trigramm in den Trainingsdaten vorkommen. Jedes Wort steht dabei so oft auf der Liste, wie es auf das entsprechende Wort-Trigramm im Text folgt. "gegen ende des" kommt insgesamt 44 Mal in unseren Trainingsdaten vor. In neun dieser Fälle folgt als nächstes Wort in den Trainingsdaten "jahres" (was den o.&nbsp;g. 20% entspricht). Entsprechend sieht das Schlüssel-Werte-Paar von "gegen ende des" so aus:

`{'gegen ende des': ['15-minütigen', '18', '19', '19', '20', 'artikels', 'beitrags', 'buchs', 'dritten', 'ersten', 'ersten', 'ersten', 'festakts', 'gesprächs', 'halbjahres', 'interviews', 'jahres', 'jahres', 'jahres', 'jahres', 'jahres', 'jahres', 'jahres', 'jahres', 'jahres', 'jahrhunderts', 'jahrzehnts', 'konzertes', 'konzerts', 'krieges', 'monats', 'ns-regimes', 'reiches', 'rennens', 'rennens', 'spiels', 'spiels', 'spiels', 'stints', 'zweiten', 'zweiten', 'zweiten', 'zweiten', 'zweiten']`

Wie genau die Wort-Trigramme berechnet werden und wie die jeweils darauffolgenden Wörter als Werte zum jeweiligen Wort-Trigramm in einem dictionary gespeichert werden, wird in den Kommentaren des folgenden Codes erläutert. Führe die Code-Zelle in jedem Fall aus, um unseren Algorithmus zu trainieren.

In [ ]:
ngrams = {} #Erstellen eines leeren dictionaries, an das unten die Schlüssel-Werte-Paare (Trigram-Liste mit darauffolgenden Wörtern) angehängt werden
n = 3 #Definieren der Länge des Wort-n-Gramms

#Iterieren über die Indizes von "words", d. h. zuerst entspricht "i" 0 (Python beginnt bei 0 zu zählen), 
#dann 1, etc., bis die Länge von "words" erreicht wurde; "tdqm" sorgt für eine Fortschrittsanzeige während der Iteration
for i in tqdm(range(len(words)-n)):
    
    #Schaffen des jeweiligen Wort-n-Gramms, indem das gegenwärtige Wort (word[i]) sowie alle weiteren Wörter bis Index i plus n zu einem
    #string zusammengefügt werden (join)
    current_ngram = " ".join(words[i:i+n])
    
    #Überprüfen, ob "current_ngram" bereits ein Schlüssel in "ngram" ist, wenn nein...
    if current_ngram not in ngrams.keys():
        
        #...Schaffen eines neuen Schlüssel-Werte-Paars mit dem jeweiligen Wort-n-Gramm als Schlüssel und zunächst leerer Liste,
        #anschließend sowie bei jedem weiteren Vorkommen des gegebenen Wort-n-Gramms wird die Liste um das jeweils darauffolgende Wort
        #erweitert (siehe letzter Schritt)
        ngrams[current_ngram] = [] 
    
    #Anfügen des Wortes, das auf das gegenwärtige Wort-n-Gramm folgt (also das Wort an Position i+n) an die Liste mit Werten zum jeweiligen Schlüssel
    ngrams[current_ngram].append(words[i+n])

Um zu überprüfen, ob das Training geklappt hat, können wir uns etwa die Werte zum Schlüssel "gegen ende des" mithilfe folgender Syntax ausgeben lassen:

In [ ]:
print(sorted(ngrams["gegen ende des"])) #Zusätzlich alphabetisch sortiert

Setz gerne andere Wort-Trigramme zwischen den Anführungszeichen ein, um die entsprechenden Werte zu erhalten. Beachte, dass es sich um exakt drei Wörter handeln muss sowie, dass Du nur Wort-Trigramme abfragen kannst, die auch in den Trainingsdaten vorkamen. Ist dies nicht der Fall, erhältst Du einen `KeyError`. 

## 3. Text generieren

Zum Schluss wollen wir unseren Algorithmus zur Textgeneration einsetzen. Dazu definieren wir mit `current_ngram` ein Trigramm, das wir dem Algorithmus als Startpunkt für den zu generierenden Text übergeben. Außerdem spezifizieren wir, wie lang der Text maximal sein soll (`len_text`). Der Algorithmus schaut nun zuerst, ob `current_ngram` im dictionary `ngrams` existiert. Wenn nicht, hat er keine Grundlage, um Folgewörter zu prognostizieren und bricht ab (`break`). Falls doch, greift er auf den Wert, der zum Schlüssel `current_ngram` in `ngrams` gespeichert ist, zu. Der Wert entspricht wie gesagt einer Liste mit Wörtern, die in den Trainingsdaten ein oder mehrere Male auf das gegebene Trigramm folgten. Nun wird **zufällig** ein Wort aus dieser Liste als nächstes Wort im generierten Text gewählt. Die Häufigkeit, mit der ein Wort in den Trainingsdaten auf ein bestimmtes Wort-Trigramm folgte und mit der es entsprechend in der Wortliste vertreten ist, korreliert natürlich mit der Wahrscheinlichkeit, mit der es nun zufällig als Folgewort gewählt wird. Abschließend werden die neuen letzten drei Wörter des generierten Textes als Wort-Trigramm definiert, das zur Generation des nächsten Folgeworts (in der nächsten Iteration) verwendet wird.

In [ ]:
current_ngram = "gegen ende des" #Definieren eines Startpunkts für den zu generierenden Text
len_text = 100 #Definieren der maximalen Länge des zu generierenden Texts
text = current_ngram #"text" beginnt mit "current_ngram"

#Wiederholen des Generationsprozesses so oft, wie "len_text" definiert
for i in range(len_text):
    
    #Überprüfen, ob "current_ngram" als Schlüssel in "ngrams" vorkommt, wenn nicht wird die Iteration abgebrochen
    if current_ngram not in ngrams.keys():
        break
        
    possible_words = ngrams[current_ngram] #Alle Folgewörter auf "current_ngram" werden "possible_words" zugewiesen
    next_word = random.sample(possible_words, 1)[0] #Mithilfe von "random" wird zufällig ein Folgewort aus "possible_words" gewählt
    text += " " + next_word #"text" wird um ein Leerzeichen und "next_word" verlängert
    text_words = text.split() #"text" wird zu Wortliste umgewandelt, um...
    current_ngram = " ".join(text_words[len(text_words)-n:len(text_words)]) #...die letzten 3 (n) Wörter als neues Trigramm "current_ngram" zuzuweisen

text = re.sub(r"(\s)(\.|,|\?|!)(\s)", r"\2\3", text) #Handling von Leerzeichen vor Interpunktion
print(text) #Finale Ausgabe des generierten Textes

Generier verschiedene Texte mit unterschiedlichen Start-Trigrammen! Machen die Texte Sinn? Wo hapert es bei der Textgeneration?

***

Ein paar Gedanken zum Schluss: Dieser Algorithmus operiert einzig und allein basierend auf Wahrscheinlichkeiten. Er hat natürlich kein Sprachverständnis, wie wir Menschen es haben. Das wird in den generierten Texten offenbar, sobald es um größere Zusammenhänge geht. Da menschliche Sprache aber so musterhaft ist, wird auf den ersten Blick oft doch erstaunlich guter Text generiert. 

ChatGPT und andere Textgenerations-KIs funktionieren übrigens im Grunde genau gleich: Sie generieren Text, der am wahrscheinlichsten zum eingebenen Prompt passt. Von der Bedeutung des Prompt ebenso wie von der Bedeutung der gelieferten Antwort versteht chatGPT rein gar nichts. 